In [1]:
# steps on this task:
# Build a FCN in Tensorflow Keras
# Testing model on the downloaded dataset
# Creating a Generator in Keras to load and process a batch of data in memory
# Training the network using variable batch dimensions

In [2]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [4]:
# upsample function utilizing tf.keras.layers.Conv2D, size=3, strides=2, 2x resolution
def upsample_2x(filters, size=2, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [5]:
def fcn_model(classes=5, drop_out_rate=0.2):
    # use dropout and bacth normalization to prevent overfitting and help to do quick convergence
    # activation layer is added to incorporate non-linearity
    
    # input layer has variable length in width and height
    input_imgs = tf.keras.layers.Input(shape=(None, None, 3))
    
    # First conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input_imgs)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.MaxPooling2D()(x)
    
    # Second conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.MaxPooling2D()(x)
    
    # Third conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool3 = tf.keras.layers.MaxPooling2D()(x)
    
    # Forth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1)(pool3)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool4 = tf.keras.layers.MaxPooling2D()(x)
    
    # Fifth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=1)(pool4)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool5 = tf.keras.layers.MaxPooling2D()(x)
    
    # build the fully connected layer using 1*1 convolutional layer
    x = tf.keras.layers.Conv2D(filters=4096, kernel_size=1, strides=1)(pool5)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    conv6 = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=classes, kernel_size=1, strides=1)(conv6)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    conv7 = tf.keras.layers.Activation('sigmoid')(x)
    
    # upsampling conv7 to 4x times and upsample pool4 to 2x times
    up_conv7 = upsample_2x(filters=classes)(upsample_2x(filters=classes)(conv7))
    up_pool4 = upsample_2x(filters=256)(pool4)
    # Concatenate two resolutions
    fuse_1 = tf.keras.layers.Concatenate(up_conv7, up_pool4, name="fuse_1")
    fuse_2 = tf.keras.layers.Concatenate(fuse_1, up_pool3, name="fuse_2")
    
    prob = upsample_2x(filters=classes)(upsample_2x(filters=classes)(upsample_2x(filters=classes)(fuse_2)))
    model = tf.keras.Model(inputs=input_imgs, outputs=prob)
    
    #print(model.summary())
    print("FCN model building completes")
    
    return model

In [6]:
#import tensorflow_datasets as tfds
#tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [1]:
from tensorflow_examples.models.pix2pix import pix2pix

228818944/228813984 [==============================] - 20s 0us/step
Downloaded and extracted at ./datasets/flower_photos
Copying images for dandelion...
Copying images for daisy...
Copying images for roses...
Copying images for sunflowers...
Copying images for tulips...
Training dataset stats:

--> Images in dandelion: 500
--> Images in daisy: 500
--> Images in roses: 500
--> Images in sunflowers: 500
--> Images in tulips: 500
Validation dataset stats:
--> Images in dandelion: 100
--> Images in daisy: 100
--> Images in roses: 100
--> Images in sunflowers: 100
--> Images in tulips: 100

AVG_IMG_HEIGHT: 326
AVG_IMG_WIDTH: 438
MIN_HEIGHT: 159
MIN_WIDTH: 143
MAX_HEIGHT: 442
MAX_WIDTH: 1024



(5, 'dataset/train', 'dataset/val', 326, 438, 2500, 500)

79
16
(32, 375, 500, 3)
(32, 5)


shape of output image:  Tensor("strided_slice_8:0", shape=(5,), dtype=float32)
FCN model building completes
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


InvalidArgumentError:  Incompatible shapes: [8,32,52,5] vs. [8,38,58,5]
	 [[node gradient_tape/model_15/tf_op_layer_fuse_1_15/BroadcastGradientArgs (defined at <ipython-input-81-00ce85375234>:14) ]] [Op:__inference_train_function_50038]

Function call stack:
train_function


2592